In [1]:
import pandas as pd
import afqinsight.nn.tf_models as nn
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from afqinsight.datasets import AFQDataset
from afqinsight.nn.tf_models import cnn_lenet, mlp4, cnn_vgg, lstm1v0, lstm1, lstm2, blstm1, blstm2, lstm_fcn, cnn_resnet
from sklearn.impute import SimpleImputer
import os.path
# Harmonization
from sklearn.model_selection import train_test_split
from neurocombat_sklearn import CombatModel
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error

import pickle
from tools import load_data, model_fit, fit_and_eval

In [2]:
X, y, site = load_data()

2022-07-26 07:18:26.723254: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 07:18:26.730289: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 07:18:26.730650: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-26 07:18:26.731570: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [3]:
model_dict = {
#   "cnn_lenet": {"model": cnn_lenet, "lr": 0.001}, 
  # "mlp4": {"model": mlp4, "lr": 0.001},
  "cnn_vgg": {"model": cnn_vgg, "lr": 0.001}
  # "lstm1v0": {"model": lstm1v0, "lr": 0.01},
  # "lstm1": {"model": lstm1, "lr": 0.01},
  # "lstm2": {"model": lstm2, "lr": 0.01},
  # "blstm1": {"model": blstm1, "lr": 0.01},
  # "blstm2": {"model": blstm1, "lr": 0.01},
  # "lstm_fcn": {"model": lstm_fcn, "lr": 0.01},
#   "cnn_resnet": {"model": cnn_resnet, "lr": 0.01}
             }

In [4]:
n_runs = 10

In [5]:
# Generated once with this code and then hard-coded:
# seeds = np.array([np.abs(np.floor(np.random.randn()*1000)) for ii in range(n_runs)], dtype=int)

seeds = np.array([484, 645, 714, 244, 215, 1503, 1334, 1576, 469, 1795])

In [6]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=None)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_1_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_1_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 72)]         0         
_________________________________________________________________
conv1d (Conv1D)              (None, 100, 64)           13888     
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 50, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 50, 128)           24704     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 50, 128)           49280     
__________________________________

2022-07-26 07:18:34.435347: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-07-26 07:18:35.069660: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-07-26 07:18:35.353762: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 07:18:35.354288: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 07:18:35.354347: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-07-26 07:18:35.355163: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-07-26 07:18:35.355262: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx co

37/37 [==============================] - 5s 50ms/step - loss: 223.3763 - mean_squared_error: 223.3763 - rmse: 14.9458 - mean_absolute_error: 6.4071 - val_loss: 10.5115 - val_mean_squared_error: 10.5115 - val_rmse: 3.2421 - val_mean_absolute_error: 2.7068
Epoch 2/1000
37/37 [==============================] - 1s 28ms/step - loss: 12.9747 - mean_squared_error: 12.9747 - rmse: 3.6020 - mean_absolute_error: 2.8371 - val_loss: 12.8979 - val_mean_squared_error: 12.8979 - val_rmse: 3.5914 - val_mean_absolute_error: 2.7118
Epoch 3/1000
37/37 [==============================] - 1s 27ms/step - loss: 11.4950 - mean_squared_error: 11.4950 - rmse: 3.3904 - mean_absolute_error: 2.6857 - val_loss: 10.2171 - val_mean_squared_error: 10.2171 - val_rmse: 3.1964 - val_mean_absolute_error: 2.6930
Epoch 4/1000
37/37 [==============================] - 1s 27ms/step - loss: 11.3072 - mean_squared_error: 11.3072 - rmse: 3.3626 - mean_absolute_error: 2.6130 - val_loss: 9.6961 - val_mean_squared_error: 9.6961 - val

In [7]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=1000)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_2_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_2_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_100 (Conv1D)          (None, 100, 64)           13888     
_________________________________________________________________
conv1d_101 (Conv1D)          (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_40 (MaxPooling (None, 50, 64)            0         
_________________________________________________________________
conv1d_102 (Conv1D)          (None, 50, 128)           24704     
_________________________________________________________________
conv1d_103 (Conv1D)          (None, 50, 128)           49280     
_______________________________

In [8]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=700)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_3_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_3_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_200 (Conv1D)          (None, 100, 64)           13888     
_________________________________________________________________
conv1d_201 (Conv1D)          (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_80 (MaxPooling (None, 50, 64)            0         
_________________________________________________________________
conv1d_202 (Conv1D)          (None, 50, 128)           24704     
_________________________________________________________________
conv1d_203 (Conv1D)          (None, 50, 128)           49280     
_______________________________

In [9]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=350)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_4_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_4_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_31 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_300 (Conv1D)          (None, 100, 64)           13888     
_________________________________________________________________
conv1d_301 (Conv1D)          (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_120 (MaxPoolin (None, 50, 64)            0         
_________________________________________________________________
conv1d_302 (Conv1D)          (None, 50, 128)           24704     
_________________________________________________________________
conv1d_303 (Conv1D)          (None, 50, 128)           49280     
_______________________________

In [10]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=175)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_5_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_5_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model_40"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_41 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_400 (Conv1D)          (None, 100, 64)           13888     
_________________________________________________________________
conv1d_401 (Conv1D)          (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_160 (MaxPoolin (None, 50, 64)            0         
_________________________________________________________________
conv1d_402 (Conv1D)          (None, 50, 128)           24704     
_________________________________________________________________
conv1d_403 (Conv1D)          (None, 50, 128)           49280     
_______________________________

In [11]:
dfs_eval = []
dfs_pred = []
for model in model_dict:
    print("##################################################")
    print("model: ", model)
    for ii in range(n_runs): 
        print("run: ", ii)
        this_eval, this_pred = fit_and_eval(
            model, 
            model_dict,
            X, 
            y, 
            site,
            random_state=seeds[ii],
            train_size=100)
        this_eval["run"] = ii
        this_pred["run"] = ii
        dfs_eval.append(this_eval)
        dfs_pred.append(this_pred)
        # Save evaluation metrics
        one_df = pd.concat(dfs_eval)
        one_df.to_csv("vgg_6_eval.csv")
        # Save predictions and test values:
        one_df = pd.concat(dfs_pred)
        one_df.to_csv("vgg_6_pred.csv")

##################################################
model:  cnn_vgg
run:  0
pooling layers: 4
Model: "model_50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 100, 72)]         0         
_________________________________________________________________
conv1d_500 (Conv1D)          (None, 100, 64)           13888     
_________________________________________________________________
conv1d_501 (Conv1D)          (None, 100, 64)           12352     
_________________________________________________________________
max_pooling1d_200 (MaxPoolin (None, 50, 64)            0         
_________________________________________________________________
conv1d_502 (Conv1D)          (None, 50, 128)           24704     
_________________________________________________________________
conv1d_503 (Conv1D)          (None, 50, 128)           49280     
_______________________________

In [12]:
# We want to try different training sample sizes: s
# (1453, 1000, 700, 350, 175)